# Dependency code

In [ ]:
import imp
import os
import sys
import numpy as np
import glob
import cmdbench

fp, pathname, description = imp.find_module('benchmark', ['../lib'])
benchmark = imp.load_module('benchmark', fp, pathname, description)

# Summarize numpy array if it has more than 10 elements
np.set_printoptions(threshold=10)

# Software versions

In [ ]:
!conda run --name mentalist mentalist -v

# Benchmark

## Input data and constants

In [ ]:
input_dir = '../data/input-files/reads'
output_dir = 'output'
db_dir = 'db/campy'
db = f'{db_dir}/campy.db'

nproc = 32

input_samples = [os.path.basename(f) for f in glob.glob(f'{input_dir}/*_1.fastq.gz')]
input_samples = [f.replace('_1.fastq.gz','') for f in input_samples]

benchmark.create_folder_if_doesnt_exist(db_dir)
benchmark.create_folder_if_doesnt_exist(output_dir)
benchmark.clean_if_exists(db_dir)
benchmark.clean_if_exists(output_dir)

sample_sizes = [1,10,20,30,40,50,60,70,80]
#sample_sizes = [3,10,15]

print(input_samples)

## Build MentaLiST database

In [ ]:
!conda run --name mentalist mentalist download_pubmlst -k 31 -s 30 -o {db_dir}/fasta --db {db}

## Benchmark functions

In [ ]:
def reset_func():
    pass

def sampling_func(sample_size):
    samples = input_samples[:sample_size]
    return samples

assemble_command = {
    "command": f"conda run --name mentalist mentalist call -o {output_dir}/%.txt --db {db} -1 {input_dir}/%_1.fastq.gz -2 {input_dir}/%_2.fastq.gz",
    "parallel_args": f"-j {nproc} -I%"
}

## Benchmark

In [ ]:
multibench_results, debug_str = benchmark.multibench.multi_cmdbench({
        "call": [assemble_command]
    },
    reset_func = reset_func, iterations = 1, sampling_func = sampling_func, sample_sizes = sample_sizes, 
    benchmark_list_to_results = benchmark.benchmark_list_to_results, active_output_print = True
)

## Save and reload results

In [ ]:
save_path = "mentalist-results.txt"

samples_per_sample_size = []
for sample_size in sample_sizes:
        samples_per_sample_size.append(input_samples[:sample_size])

benchmark.multibench.save_multibench_results(multibench_results, samples_per_sample_size, save_path)
multibench_results, samples_per_sample_size = benchmark.multibench.read_multibench_results(save_path)
print(samples_per_sample_size)

# Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

In [ ]:
# Indexing Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "call")